In [76]:
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import os
import string
import copy
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aziz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aziz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [77]:
title = "comp.graphics"
os.chdir("C:/mini_newsgroups")
paths = []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'):
    for i in filenames:
        paths.append(str(dirpath)+str("\\")+i)
        
print(dirpath)

C:\mini_newsgroups/comp.graphics/


In [78]:
paths[0]

'C:\\mini_newsgroups/comp.graphics/\\37916'

In [79]:
len(paths)

100

# Exercise 1: What does the following representation mean?{23: [3, [0, 12, 28]]}
23 = posting list

3 = frequency post index

0, 12, 28 = position

# Exercise 2: Write a set of functions that is able to remove: stop words, punctuation, apostrophe, single characters, and header. In addition, you have to convert words into lowercase, stem the inter words and convert numbers to its equivalent words. (Hint: use the same functions that are in Lab06: unigram indexing)

In [80]:
def preprocess(data, query):
    if not query:
        data = remove_header(data)
        data = remove_single_characters(data)
        data = remove_apostrophe(data)
        data = convert_numbers(data)
        data = stemming(data)
        data = convert_lower_case(data)
        data = remove_punctuation(data)
        data = remove_stop_words(data)
    return data

In [81]:
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

In [82]:
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [83]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [84]:
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

In [85]:
def stemming(data):
    stemmer= PorterStemmer()
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

In [86]:
def convert_lower_case(data):
    return np.char.lower(data)

In [87]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], '')
        data = np.char.replace(data, " ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [88]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [89]:
postings = pd.DataFrame()
frequency = pd.DataFrame()
doc = 0
for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)
    if doc%100 == 0:
        print(doc)
    tokens = word_tokenize(str(preprocessed_text))
    
    pos = 0
    for token in tokens:
        if token in postings:
            p = postings[token][0]
            
            k = [a[0] for a in p]
            if doc in k:
                for a in p:
                    if a[0] == doc:
                        a[1].add(pos)
            else:
                p.append([doc,{pos}])
                frequency[token][0] += 1
        else:
            postings.insert(value=[[[doc, {pos}]]], loc=0, column=token)
            frequency.insert(value=[1], loc=0, column=token)
            
        pos += 1
    doc += 1

0


In [90]:
postings

,fontmong,lazer,rip,smirk,shop,nasti,undercut,ventur,pete,truetyp,...,simpl,sipp,call,routin,render,librari,describ,file,got,recent
0,"[[99, {100}]]","[[99, {94}]]","[[99, {83}]]","[[99, {74}]]","[[99, {73}]]","[[99, {65}]]","[[99, {61}]]","[[99, {60}]]","[[99, {39}]]","[[99, {57, 30, 78}]]",...,"[[0, {9}], [1, {49}], [15, {8}], [17, {4146, 4...","[[0, {8, 29}]]","[[0, {7}], [5, {42}], [10, {72}], [17, {352, 3...","[[0, {33, 6}], [5, {128, 119}], [10, {13}], [1...","[[0, {32, 5}], [1, {27}], [17, {796}], [46, {1...","[[0, {4, 31}], [1, {16}], [5, {13}], [17, {409...","[[0, {3}], [17, {1233}], [64, {2270}], [65, {4...","[[0, {2}], [5, {147, 165}], [7, {17, 30}], [8,...","[[0, {1}], [29, {54}], [33, {60}], [37, {111}]...","[[0, {0}], [5, {164}], [13, {0}], [17, {5175}]..."


In [91]:
frequency

,fontmong,lazer,rip,smirk,shop,nasti,undercut,ventur,pete,truetyp,...,simpl,sipp,call,routin,render,librari,describ,file,got,recent
0,1,1,1,1,1,1,1,1,1,1,...,7,1,9,9,12,14,6,25,12,7


# Exercise3: explore posting list for a term and its frequency. Explain the output.
ventur frequency post index is 99 and position is 60 and it frequency is 1

In [92]:
postings["ventur"][0]

[[99, {60}]]

In [93]:
frequency["ventur"][0]

1

In [94]:
postings["truetyp"][0]

[[99, {30, 57, 78}]]

In [95]:
frequency["truetyp"][0]

1

In [96]:
postings["call"][0]

[[0, {7}],
 [5, {42}],
 [10, {72}],
 [17, {103, 245, 352, 3043, 3089, 4409, 5213}],
 [18, {41}],
 [64,
  {134,
   169,
   734,
   827,
   860,
   1356,
   1893,
   3602,
   4260,
   4269,
   4453,
   4457,
   4461,
   4997,
   5403,
   5418,
   5624,
   5950,
   6331,
   6478,
   6588,
   6760}],
 [66, {90}],
 [74, {4, 35}],
 [91, {3252, 3270, 3847, 4241, 6104}]]

In [97]:
frequency["call"][0]

9

In [98]:
postings.to_pickle(title + "_positional_postings")
frequency.to_pickle(title + "_positional_frequency")

In [99]:
postings = pd.read_pickle(title + "_positional_postings")

In [100]:
frequency = pd.read_pickle(title + "_positional_frequency")

# Exercise 4: What is the posting list of “not in”?
here because "not in" does not exist i got this error

In [101]:
postings["not in"][0]

KeyError: 'not in'

# Exercise5. For the whole collection given by theinstructor, build the positional index and find frequency for each term appears in collection!

In [102]:
def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",frequency[preprocessed_word][0])
    print("Postings List:",postings[preprocessed_word][0])

In [103]:
def get_positions(posting_values, doc):
    for posting_value in posting_values:
        if posting_value[0] == doc:
            return posting_value[1]
    return {}

In [104]:
def gen_init_set_matchings(word):
    init = []
    word_postings = postings[word][0]
    for word_posting in word_postings:
        for positions in word_posting[1]:
            init.append((word_posting[0], positions))
    return init

In [105]:
def match_positional_index(init, b):
    matched_docs = []
    for p in init:
        doc = p[0]
        pos = p[1]

        count = 0

        for k in b:
            pos = pos+1
            k_pos = postings[k][0]
            docs_list = [z[0] for z in k_pos]
            if doc in docs_list:
                doc_positions = get_positions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(b):
                matched_docs.append(p[0])
    return set(matched_docs)   

In [106]:
def run_query(query):
    processed_query = preprocess(query, True)
    print(processed_query)

    query_tokens = word_tokenize(str(processed_query))
    print(query_tokens)
    
    if len(query_tokens)==1:
        print("Total Document Mathces", [a[0] for a in postings[query][0]])
        return [a[0] for a in postings[query][0]]
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(init_word)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    return total_matched_docs

In [107]:
get_word_postings("simpl")

simpl
Frequency: 7
Postings List: [[0, {9}], [1, {49}], [15, {8}], [17, {4146, 4660, 996, 4458}], [37, {762}], [64, {5502}], [91, {5633, 1499, 6065, 1647}]]


In [108]:
query = "ventur  undercut"
lists = run_query(query)

ventur  undercut
['ventur', 'undercut']
Total Document Matches: {99}


In [109]:
query = postings
lists = run_query(query)

        fontmong         lazer           rip         smirk          shop  \
0  [[99, {100}]]  [[99, {94}]]  [[99, {83}]]  [[99, {74}]]  [[99, {73}]]   

          nasti      undercut        ventur          pete  \
0  [[99, {65}]]  [[99, {61}]]  [[99, {60}]]  [[99, {39}]]   

                truetyp  ...  \
0  [[99, {57, 30, 78}]]  ...   

                                               simpl            sipp  \
0  [[0, {9}], [1, {49}], [15, {8}], [17, {4146, 4...  [[0, {8, 29}]]   

                                                call  \
0  [[0, {7}], [5, {42}], [10, {72}], [17, {352, 3...   

                                              routin  \
0  [[0, {33, 6}], [5, {128, 119}], [10, {13}], [1...   

                                              render  \
0  [[0, {32, 5}], [1, {27}], [17, {796}], [46, {1...   

                                             librari  \
0  [[0, {4, 31}], [1, {16}], [5, {13}], [17, {409...   

                                             describ  \
0  [[0